# Aligning structures and trajectories

Last updated: November 2019

Minimum version of MDAnalysis: 0.16.0

In this tutorial we:
* use `align.alignto` to align a structure to another
* use `align.AlignTraj` to align a trajectory and write it to a file

In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.tests.datafiles import CRD, PSF, DCD, DCD2
import nglview as nv

_ColormakerRegistry()

## Loading files

The test files we will be working with here are trajectories of a adenylate kinase (AdK), a phosophotransferase enzyme. The trajectories sample a transition from a closed to an open conformation. If you have NGLView installed or are viewing this notebook online, you can play the trajectories below and watch the transition.

In [2]:
adk_open = mda.Universe(CRD, DCD2)
nv.show_mdanalysis(adk_open)

NGLWidget(max_frame=101)

/Users/lily/anaconda3/envs/mda0160/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  "".format(attrname, default))
/Users/lily/anaconda3/envs/mda0160/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  "".format(attrname, default))
/Users/lily/anaconda3/envs/mda0160/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  "".format(attrname, default))


In [3]:
adk_closed = mda.Universe(PSF, DCD)
nv.show_mdanalysis(adk_closed.atoms)

/Users/lily/anaconda3/envs/mda0160/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'tempfactors' Using default value of '0.0'
  "".format(attrname, default))


NGLWidget(max_frame=97)

Currently, the proteins are not aligned to each other. The difference becomes even more obvious when the closed conformation is compared to the open. Below, we set `adk_open` to the last frame and see the relative positions of each protein in a merged Universe.

In [4]:
adk_open.trajectory[-1] # last frame
merged = mda.Merge(adk_open.atoms, adk_closed.atoms)
nv.show_mdanalysis(merged)

NGLWidget()

## Aligning a structure with align.alignto

`alignto` aligns the mobile AtomGroup to the target AtomGroup by minimising the RMSD. It returns (old_rmsd, new_rmsd)

In [5]:
align.alignto(adk_open,  # mobile
              adk_closed,  # reference
              select='name CA') # selection to operate on

(21.712154435976014, 6.817293751703893)

In [6]:
nv.show_mdanalysis(mda.Merge(adk_open.atoms, adk_closed.atoms))

NGLWidget()

However, positions are set temporarily. If we flip to the first frame of `adk_open` and back to the last frame, we can see that it has returned to its original location.

In [7]:
adk_open.trajectory[0] # set to first frame
adk_open.trajectory[-1] # set to last frame
nv.show_mdanalysis(mda.Merge(adk_open.atoms, adk_closed.atoms))

NGLWidget()

You can save the aligned positions by writing them out to a PDB file and creating a new Universe.

In [8]:
align.alignto(adk_open, adk_closed, select='name CA')
adk_open.atoms.write('aligned.pdb')
nv.show_mdanalysis(mda.Universe('aligned.pdb'))

NGLWidget()

## Aligning a trajectory with AlignTraj

While `align.alignto` aligns structures, or a frame of a trajectory, `align.AlignTraj` efficiently aligns an entire trajectory to a reference. 

In [9]:
align.AlignTraj(adk_closed, adk_open, select='name CA', filename='aligned.dcd').run()

In [10]:
aligned = mda.Universe(PSF, 'aligned.xtc')
nv.show_mdanalysis(aligned)

/Users/lily/anaconda3/envs/mda0160/lib/python3.7/site-packages/MDAnalysis/lib/mdamath.py:259: RuntimeWarning: invalid value encountered in double_scalars
  alpha = np.rad2deg(np.arccos(np.dot(y, z) / (ly * lz)))
/Users/lily/anaconda3/envs/mda0160/lib/python3.7/site-packages/MDAnalysis/lib/mdamath.py:260: RuntimeWarning: invalid value encountered in double_scalars
  beta = np.rad2deg(np.arccos(np.dot(x, z) / (lx * lz)))
/Users/lily/anaconda3/envs/mda0160/lib/python3.7/site-packages/MDAnalysis/lib/mdamath.py:261: RuntimeWarning: invalid value encountered in double_scalars
  gamma = np.rad2deg(np.arccos(np.dot(x, y) / (lx * ly)))
/Users/lily/anaconda3/envs/mda0160/lib/python3.7/site-packages/MDAnalysis/lib/mdamath.py:264: RuntimeWarning: invalid value encountered in greater
  if np.all(box > 0.0) and alpha < 180.0 and beta < 180.0 and gamma < 180.0:


NGLWidget(max_frame=99)

In [33]:
aligned.segments.segids = ['Aligned']
adk_open.segments.segids = ['Open']

In [34]:
merged2 = mda.Merge(aligned.atoms, adk_open.atoms)
nv.show_mdanalysis(merged2)

NGLWidget()

`MDAnalysis.Merge` does not automatically load coordinates for a trajectory. We can do this ourselves.

In [37]:
from MDAnalysis.analysis.base import AnalysisFromFunction
import numpy as np
from MDAnalysis.coordinates.memory import MemoryReader

def copy_coords(ag):
    return ag.positions.copy()

aligned_coords = AnalysisFromFunction(copy_coords, aligned.atoms).run().results

print(aligned_coords.shape)

(100, 3341, 3)


In [38]:
adk_coords = adk_open.atoms.positions.copy()
adk_coords.shape

(3341, 3)

In [39]:
adk_traj_coords = np.stack([adk_coords] * 100)
adk_traj_coords.shape

(100, 3341, 3)

In [40]:
merged_coords = np.hstack([aligned_coords, adk_traj_coords])
merged2.load_new(merged_coords, format=MemoryReader)

<Universe with 6682 atoms>

In [41]:
nv.show_mdanalysis(merged2)

NGLWidget(max_frame=99)